# Project 9: Data Collection

**Name**: Catelyn Clevenger

**Collaborators**: Professor Wirfs-Brock

**Sources**:


* I used W3 schools to review the requests function.(https://www.w3schools.com/python/ref_requests_get.asp).

* The "Steam Games Dataset" on Kaggle and SteamDB got me started with exploring the possible methods I could use to find my data(https://www.kaggle.com/datasets/fronkongames/steam-games-dataset/data,  https://steamdb.info/faq/#how-are-we-getting-this-information).

* I read this documentation for the official Steam Web API: https://developer.valvesoftware.com/wiki/Steam_Web_API#Interfaces_and_method.

*  This set of links helped me understand the officially undocumented Steam storefront API: https://zuplo.com/blog/2024/10/04/what-is-the-steam-web-api, https://github.com/Revadike/InternalSteamWebAPI/wiki.

*   I asked Chat GPT for advice on dealing with JavaScript while webscraping, where to find documentation for the unofficial Steam Storefront API, and how to avoid rate limiting.



## **Goal**:  From Steam, I want to create a dataset that will help me explore how sound design affects players' gameplay and overall experience.

## **Experimentation**: Choosing a Collection Method

### 1) The official Steam API

After getting access to the Steam API, I realized it did not have most of the data I wanted to collect.  It included data used to identify and request information about apps on the Steam storefront (app IDs and names).  The rest of the information was about apps' news and user data (not including user reviews), which I was not planning to use for this project.

### 2) Webscraping the Steam Site

Webscraping the Steam website was the first time I tried to deal with JavaScript while webscraping.  I consulted ChatGPT about possible methods to deal with JavaScript.  ChatGPT recommended that I use Selenium or find the API Steam used to load its data.  I decided to attempt the second option since Chat GPT warned me that using Selenium to collect data would be slow.

While inspecting the webpage, I observed that the data I wanted was tied to fetch and XHR events.  Looking through all the instances of fetch and XHR, I saw two different domains: store.steampowered.com and api.steampowered.com.  I knew that api.steampowered.com was Steam's official API, so I wondered if Steam was using another API for its store page.  After asking ChatGPT, I was told about the unofficial Steam Storefront API.

### 3) The unofficial Steam Storefront API

This API had the rest of the data I wanted to collect that the official Steam API did not provide.  It allowed me to request information about apps found on their Steam store pages (reviews, links to related apps and bundles, and game genres).

### Method Decision:

I collected my data using the official Steam API and the unofficial Steam Storefront API.  I used the official Steam API to return the IDs and names of all the available apps on the website, and I used the Steam Storefront API to find what game soundtracks are from, reviews, and game genres.

## **Data Collection**: Steam APIs

In [ ]:
# First, I will import the necessary packages.

import pandas as pd
import requests
import time
import random

In [ ]:
# The app list url returns a list of all the apps listed for sale on Steam (including Soundtracks)
# The app details url includes basic info such as the game's name, its price, the developers and publishers
# Also, the app details url provides the app_id of the main game.  This is very important for finding the game a soundtrack is from!

steam_app_list_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
steam_app_details_url = "https://store.steampowered.com/api/appdetails"
min_reviews = 20      # I want the soundtracks and games to have a minimum of 20 reviews
max_rows = 500     # This limits how many soundtracks and games to save after filtering. (max_rows * 2 = number of products)
csv_filename = "game_soundtrack_steamreviews.csv"

In [ ]:
# This function returns the app ids and names of all the Steam games listed for sale
def get_all_apps():
    print("Fetching Steam app list...")
    response = requests.get(steam_app_list_url)
    response.raise_for_status()
    return response.json()['applist']['apps']

In [ ]:
# This function filters through a list of apps and returns a list of soundtracks
# by searching for "soundtrack" in each app's title
def filter_soundtracks(apps):
    return [app for app in apps if "soundtrack" in app['name'].lower()]

In [ ]:
# This function returns a dictionary of all the basic details that are found on an app's store page.
# However, this does not include the app's reviews.
def get_app_details(appid):
    params = {'appids': appid}
    response = requests.get(steam_app_details_url, params=params)
    if response.status_code != 200:
        return None
    data = response.json()
    if not data.get(str(appid), {}).get("success"):
        return None
    return data[str(appid)]["data"]

In [ ]:
# This function returns the total number of reviews for a game and the number of positive reviews.
# I will use this function to calculate the percentage of positive reviews for a game.
def get_review_info(appid):
    url = f"https://store.steampowered.com/appreviews/{appid}"
    params = {
        'json': 1,
        'filter': 'summary',
        'language': 'all'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        return None
    data = response.json()
    total = data['query_summary']['total_reviews']
    positive = data['query_summary']['total_positive']
    return total, positive

In [ ]:
#Now, I will find my list of soundtracks which I will filter through.
all_apps = get_all_apps()
soundtracks = filter_soundtracks(all_apps)
random.shuffle(soundtracks)  # Shuffles to get a random sample of soundtracks
print(len(soundtracks))
#Interestingly, the length of soundtracks changes every time I tested my get_all_apps and my filter_soundtracks method

Fetching Steam app list...
9112


In [ ]:
# Looking at the unordered appids, random.shuffle effectively randomized the order of soundtracks
print(soundtracks[0])
print(soundtracks[1])

{'appid': 285550, 'name': 'Dominique Pamplemousse: Soundtrack & Sheet Music'}
{'appid': 492140, 'name': 'Dead by Daylight: Original Soundtrack'}


In [ ]:
# Before I create my csv, I will check to see whether get_app_details and get_review_info returns what I am looking for.
test_details = get_app_details(311690)
print(test_details['name'], test_details['genres'])
print(test_details['genres'][0]['description'])
test_review_info = main_game_review_info = get_review_info(311690)
print(test_review_info)

#They do!
#For games with multiple genres, I am planning on selecting the first genre listed

Enter the Gungeon [{'id': '1', 'description': 'Action'}, {'id': '25', 'description': 'Adventure'}, {'id': '23', 'description': 'Indie'}]
Action
(75067, 71749)


In [ ]:
results = []

for app in soundtracks:
    # Stops the loop once data on 500 soundtracks and 500 games is stored in results
    # I am capping my csv file to 500 rows due to the amount of time it takes to make the csv
    if len(results) >= max_rows:
        break
    appid = app['appid']
    name = app['name']

    # Ensures that get_review_info returned something
    review_info = get_review_info(appid)
    if not review_info:
        continue # Continue jumps to the next app in soundtracks

    # Ensures that the app has 20 or more reviews
    total_reviews, positive_reviews = review_info
    if total_reviews < min_reviews:
        continue

    # Calculates the percentage of positive reviews
    percent_positive = round(positive_reviews / total_reviews * 100, 2)
    print(f"{name} - {total_reviews} reviews, {percent_positive}% positive")

    # Variables to store available information on the main game
    details = get_app_details(appid)
    main_game_id = None
    main_game_name = None
    main_game_genre = None
    main_game_reviews = None
    main_game_positive = None

    # Checks to see if the soundtracks' page includes information about the main game it comes from
    if not(details and 'fullgame' in details):
      continue
    main_game_id = details['fullgame'].get('appid')
    main_game_name = details['fullgame'].get('name')

    # Finds the main game's genre, reviews, and the percentage of positive reviews if available
    # If they are not available, the sound track and main game will NOT be added to the csv
    if not main_game_id:
      continue
    main_game_review_info = get_review_info(main_game_id)
    main_game_details = get_app_details(main_game_id)

    if not main_game_details or not main_game_review_info:
      continue

    main_game_genre = main_game_details['genres'][0]['description']


    mg_total, mg_positive = main_game_review_info
    # The main game is also required to have a minimum of 20 reviews.
    if mg_total < min_reviews:
      continue
    main_game_reviews = mg_total
    main_game_positive = round(mg_positive / mg_total * 100, 2)

    print(f"{main_game_name} - {main_game_reviews} reviews, {main_game_positive}% positive")

    # Each dictionary in results will represent one row in the csv
    results.append({
        'soundtrack_appid': appid,
        'soundtrack_name': name,
        'soundtrack_reviews': total_reviews,
        'soundtrack_positive_percentage': percent_positive,
        'main_game_appid': main_game_id,
        'main_game_name': main_game_name,
        'main_game_reviews': main_game_reviews,
        'main_game_positive_percentage': main_game_positive
        })

    time.sleep(random.uniform(0.75, 1.25))  # Avoid rate-limiting by slowing the requests down. The randomness will hopefully prevent me from getting flagged as a bot.

df = pd.DataFrame(results)
df.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"\nSaved {len(results)} results to {csv_filename}")

Dead by Daylight: Original Soundtrack - 315 reviews, 77.14% positive
Dead by Daylight - 575923 reviews, 79.03% positive
eden* Soundtrack - 28 reviews, 82.14% positive
eden* - 3557 reviews, 98.09% positive
ASTRONEER (Original Soundtrack) - 109 reviews, 85.32% positive
ASTRONEER - 109874 reviews, 92.24% positive
Descenders Soundtrack - 40 reviews, 80.0% positive
Descenders - 15467 reviews, 95.27% positive
Age of Empires IV Digital Soundtrack - 49 reviews, 79.59% positive
Age of Empires IV: Anniversary Edition - 60200 reviews, 86.31% positive
Rain World: Downpour - Soundtrack - 36 reviews, 94.44% positive
Rain World - 33757 reviews, 94.69% positive
Quantum Break - Original Game Soundtrack - 25 reviews, 56.0% positive
Quantum Break - 20890 reviews, 86.4% positive
Rust Soundtrack - 153 reviews, 81.7% positive
Rust - 1013088 reviews, 87.67% positive
Ancestors Legacy - Digital Soundtrack - 28 reviews, 92.86% positive
Ancestors Legacy - 5572 reviews, 79.88% positive
Hades Original Soundtrack -

KeyError: 'total_reviews'

In [ ]:
# Uh oh! After 47 minutes of execution, an error stopped the program.
# Luckily, 470 rows of data were already collected, so I will save what I have to a csv.
print(len(results))
print(results[len(results) - 1])

470
{'soundtrack_appid': 2182230, 'soundtrack_name': "Otaku's Adventure Soundtrack", 'soundtrack_reviews': 31, 'soundtrack_positive_percentage': 96.77, 'main_game_appid': '937170', 'main_game_name': "Otaku's Adventure", 'main_game_reviews': 7575, 'main_game_positive_percentage': 94.86}


In [ ]:
df = pd.DataFrame(results)
df.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"\nSaved {len(results)} results to {csv_filename}")


Saved 470 results to game_soundtrack_steamreviews.csv


In [ ]:
# After looking at the csv,  I realized it was missing a very important column: the genre of the main game.
# Instead of starting over, I'm going to make a csv with two columns: "main_game_appid" and "main_game_genre".
# Later, I will join my two csvs by their "main_game_id" column.

df_reviews = pd.read_csv('/content/game_soundtrack_steamreviews.csv')
genre_results = []

for game_id in df_reviews['main_game_appid']:
  game_details = get_app_details(game_id)
  try:
    genre = game_details['genres'][0]['description']
  except (KeyError, IndexError, TypeError):
    genre = "Unknown"
  print(f"{game_id}, {genre}")
  genre_results.append({'main_game_appid' : game_id, 'main_game_genre' : genre})
  time.sleep(random.uniform(0.75, 1))

df_genre = pd.DataFrame(genre_results)



381210, Action
315810, Adventure
361420, Adventure
681280, Action
1466860, Strategy
312520, Action
474960, Action
252490, Action
620590, Strategy
1145360, Action
495420, Action
1366540, Indie
692850, Action
524220, Action
1472560, Adventure
1286350, Action
1223810, Casual
844590, Simulation
893680, Action
406970, Adventure
615530, Adventure
1057090, Action
2161700, Adventure
673880, Strategy
1533420, Action
881100, Action
1147550, Adventure
1347780, Action
517710, Racing
882710, Action
261550, Action
406970, Adventure
303310, Adventure
972660, Adventure
632360, Action
1145350, Action
557340, Action
418240, Indie
1794680, Action
746850, Adventure
481510, Adventure
1672970, Action
383870, Adventure
793460, Indie
788100, Action
1160220, Adventure
486310, Adventure
867210, Adventure
420290, Action
237930, Action
292030, RPG
1230140, Adventure
960170, Action
257850, Action
1817230, Action
692850, Action
555220, Adventure
1384160, Action
1748620, Action
1122720, Action
396900, Action
1182620

In [ ]:
#Now, I will merge the two data frames by their 'main_game_appid' columns

df_merged = df_reviews.merge(df_genre, on='main_game_appid', how='left')

In [ ]:
print(df_merged.shape)
print(df_merged.head())

(912, 9)
   soundtrack_appid                        soundtrack_name  \
0            492140  Dead by Daylight: Original Soundtrack   
1           1266580                       eden* Soundtrack   
2            571010        ASTRONEER (Original Soundtrack)   
3            571010        ASTRONEER (Original Soundtrack)   
4           1079460                  Descenders Soundtrack   

   soundtrack_reviews  soundtrack_positive_percentage  main_game_appid  \
0                 315                           77.14           381210   
1                  28                           82.14           315810   
2                 109                           85.32           361420   
3                 109                           85.32           361420   
4                  40                           80.00           681280   

     main_game_name  main_game_reviews  main_game_positive_percentage  \
0  Dead by Daylight             575923                          79.03   
1             eden*        

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   soundtrack_appid                912 non-null    int64  
 1   soundtrack_name                 912 non-null    object 
 2   soundtrack_reviews              912 non-null    int64  
 3   soundtrack_positive_percentage  912 non-null    float64
 4   main_game_appid                 912 non-null    int64  
 5   main_game_name                  912 non-null    object 
 6   main_game_reviews               912 non-null    int64  
 7   main_game_positive_percentage   912 non-null    float64
 8   main_game_genre                 912 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 64.3+ KB


In [ ]:
#Now, I will save the merged CSV
df_merged.to_csv("steam_reviews.csv", index=False, encoding='utf-8')

## Reflection:

This process taught me that correcting mistakes is much more demanding (time and resource-wise) when automating API requests.  I also learned that time can be a big restraint when creating your own dataset.

In the future, I want to try webscraping with Selenium to see how my data collection process changes.